<a href="https://colab.research.google.com/github/Amagnum/2D-shapes-Recognition-using-Guassian-based-probabilistic-Learning/blob/master/3D_Ken_Burns_multiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D Ken Burns
This is a reference implementation of 3D Ken Burns Effect from a Single Image [1] using PyTorch. Given a single input image, it animates this still image with a virtual camera scan and zoom subject to motion parallax. Should you be making use of our work, please cite our paper [1].

<a href="https://arxiv.org/abs/1909.05483" rel="Paper"><img src="http://content.sniklaus.com/kenburns/paper.jpg" alt="Paper" width="100%"></a>


> Colab created by [Manuel Romero](https://twitter.com/mrm8488)

### 1. Clone the project

In [1]:
!git clone https://github.com/sniklaus/3d-ken-burns.git

Cloning into '3d-ken-burns'...
remote: Enumerating objects: 202, done.
remote: Total 202 (delta 0), reused 0 (delta 0), pack-reused 202
Receiving objects: 100% (202/202), 345.75 KiB | 750.00 KiB/s, done.
Resolving deltas: 100% (126/126), done.


### 2. Download pretrained models

In [2]:
!wget -O ./3d-ken-burns/models/disparity-estimation.pytorch http://content.sniklaus.com/kenburns/network-disparity.pytorch
!wget -O ./3d-ken-burns/models/disparity-refinement.pytorch http://content.sniklaus.com/kenburns/network-refinement.pytorch
!wget -O ./3d-ken-burns/models/pointcloud-inpainting.pytorch http://content.sniklaus.com/kenburns/network-inpainting.pytorch

--2022-10-22 17:24:08--  http://content.sniklaus.com/kenburns/network-disparity.pytorch
Resolving content.sniklaus.com (content.sniklaus.com)... 88.198.95.97
Connecting to content.sniklaus.com (content.sniklaus.com)|88.198.95.97|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 357438463 (341M) [application/octet-stream]
Saving to: ‘./3d-ken-burns/models/disparity-estimation.pytorch’

./3d-ken-burns/mode 100%[===================>] 340.88M  14.8MB/s    in 25s     

2022-10-22 17:24:34 (13.6 MB/s) - ‘./3d-ken-burns/models/disparity-estimation.pytorch’ saved [357438463/357438463]

--2022-10-22 17:24:34--  http://content.sniklaus.com/kenburns/network-refinement.pytorch
Resolving content.sniklaus.com (content.sniklaus.com)... 88.198.95.97
Connecting to content.sniklaus.com (content.sniklaus.com)|88.198.95.97|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2438928 (2.3M) [application/octet-stream]
Saving to: ‘./3d-ken-burns/models/disparity-ref

In [3]:
!pip install chainer moviepy gevent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 27.4 MB/s 
     |████████████████████████████████| 5.8 MB 28.5 MB/s 
     |████████████████████████████████| 254 kB 67.5 MB/s 
  Created wheel for chainer: filename=chainer-7.8.1-py3-none-any.whl size=967740 sha256=5e26dd5db1323be15e0eda7d015315dc00bc338483785a70b86dbcedf08a092a
  Stored in directory: /root/.cache/pip/wheels/c8/6a/6f/fd563166cc597e5206e375ea074ea836e5db5dd58421215672
Successfully built chainer


In [4]:
import chainer
chainer.print_runtime_info()

Platform: Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
Chainer: 7.8.1
ChainerX: Not Available
NumPy: 1.21.6
CuPy:
  OS                           : Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
  Python Version               : 3.7.15
  CuPy Version                 : 11.0.0
  CuPy Platform                : NVIDIA CUDA
  NumPy Version                : 1.21.6
  SciPy Version                : 1.7.3
  Cython Build Version         : 0.29.24
  Cython Runtime Version       : 0.29.32
  CUDA Root                    : /usr/local/cuda
  nvcc PATH                    : /usr/local/cuda/bin/nvcc
  CUDA Build Version           : 11070
  CUDA Driver Version          : 11020
  CUDA Runtime Version         : 11020
  cuBLAS Version               : 11401
  cuFFT Version                : 10401
  cuRAND Version               : 10203
  cuSOLVER Version             : (11, 1, 0)
  cuSPARSE Version             : 11401
  NVRTC Version                : (11, 2)
  Thrust Version               : 101500
  CUB Build

In [5]:
import os
os.environ['CUDA_HOME'] = "/usr/local/cuda"

In [6]:
cd /content/3d-ken-burns

/content/3d-ken-burns


In [7]:
!rm images/README.md
!mkdir results

### 3. Upload several a pictures from your filesystem

In [14]:
from google.colab import files

uploaded = files.upload()
for filename in uploaded.keys():
  !mv ./$filename ./images/$filename

Saving ada.png to ada.png


In [11]:
!sudo pip3 install imageio==2.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 28.2 MB/s 
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303885 sha256=356936ba353a3459cdffea805162d5b606ce0d0b7bd75c90fa1d1be6f072e400
  Stored in directory: /root/.cache/pip/wheels/46/20/07/7bb9c8c44e6ec2efa60fd0e6280094f53f65f41767ef69a5ee
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0


### 4. Apply (bulk) 3D Ken Burns Effect

In [17]:
!for image in ./images/*; do python autozoom.py --in $image --out ./results/$(basename $image | cut -f1 -d '.').mp4; done

[MoviePy] >>>> Building video ./results/ada.mp4
[MoviePy] Writing video ./results/ada.mp4
100% 150/150 [00:05<00:00, 26.83it/s]
[MoviePy] Done.
[MoviePy] >>>> Video ready: ./results/ada.mp4 

[MoviePy] >>>> Building video ./results/doublestrike.mp4
[MoviePy] Writing video ./results/doublestrike.mp4
100% 150/150 [00:05<00:00, 27.93it/s]
[MoviePy] Done.
[MoviePy] >>>> Video ready: ./results/doublestrike.mp4 



#### Now you have all your videos under the folder **results** and you can *manually* right click and download them.

### If you want to *automatically* download all the videos run the following cell:

In [18]:
for filename in os.listdir("./results"):
  extension = os.path.splitext(filename)[-1].lower()
  if extension == ".mp4":
    files.download("./results/" + filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
!python depthestim.py --in ./images/doublestrike.jpg --out ./depthestim.npy


�NUMPY v {'descr': '<f4', 'fortran_order': False, 'shape': (768, 1024), }                                                     
F�F��F��FV�F�F��F�F�FU�F��FZ�F��F@�F|FFn
F�	F]�FtoFd�Fy
TF�"F��F:F��FY�FVlFzF�UF��F��F�FyHF�tF��F�F\�F�Fk#F��F�F#F�F��Fe FR�Fh�F��F1F��F�XF��F]�F�FL�FoZFW�F��FΗF�oF�F�FO2F�FH�F�dF��F��F}FK�F/F	F2,F+�F�F�F�F�bFXBFB:F��F��FKF�XF��F��FK�F:mF\�F��F;�F��F�UF}�F��F��F>�F�KF��F֏F��F��F��F �FQCF?�F��FyzFJF1F�FfF��F1�F�FǷF� F�,F`�F�F��F~�F��F�%F&F��F�F��FH�FN
F?PF��FnF��F��F�F��Fv�F�F�PFP�F��F:�F�#FA�F�'FSF�FqF��F��F��F�bF��F�{F(F�ZF){FR*F F�oF�F�Ff�F}vFމFr�F�F��F�mFՓF�F:�F	�F|�F��F�fF�RF�pF��F*�F�pF�FP�F	�F�F�Fk@F��F�SFF�FUOFd�F8�F�F%FFl0F+�F޲FUCF FbFr�F�F�F��F��F��FH�FYF�OF�eF��F�
F��FP�FF�F�zF

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 Did you enjoy? Buy me a coffe :)
 
[![ko-fi](https://www.ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Y8Y3VYYE)